## SIM Training by Stein's Method

In [3]:
import numpy as np
from matplotlib import pylab as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer, mean_squared_error

from pysim import SimRegressor

## pySIM - A python package for Sparse SIM 

**To install**:
    
```sheel
    pip install git+https://github.com/Zebinyang/pysim.git
```

Note pysim will call the R package fps (https://github.com/vqv/fps) using rpy2 interface. 

**Usage**

```python
from pysim import SimRegressor
clf = SimRegressor(method="first_order", reg_lambda=0.1, spline="smoothing_spline", reg_gamma=10, knot_num=20, knot_dist="uniform", degree=2, random_state=0)
## clf.fit(x, y)
```

**Hyperparameters**

- method: the base method for estimating the projection coefficients in sparse SIM. default="first_order"

        "first_order": First-order Stein's Identity via sparse PCA solver

        "second_order": Second-order Stein's Identity via sparse PCA solver

        "first_order_thres": First-order Stein's Identity via hard thresholding (A simplified verison)        
    
- reg_lambda: The regularization strength of sparsity of beta. default=0.1, from 0 to 1 

- spline: The type of spline for fitting the curve. default="smoothing_spline"
        
        "smoothing_spline": Smoothing spline

        "p_spline": P-spline

        "mono_p_spline": P-spline with monotonic constraint
        
        "a_spline": Adaptive B-spline

- reg_gamma: The regularization strength of the spline algorithm. default=0.1.

        For spline="smoothing_spline", it ranges from 0 to 1 
        
        For spline="p_spline","mono_p_spline" or "a_spline", it ranges from 0 to $+\infty$.

- degree: The order of the spline basis, not used in "smoothing_spline". default=2

- knot_num: The number of knots. default=20

- knot_dist: The method of specifying the knots. default="uniform"

        "uniform": uniformly over the domain
        
        "quantile": uniform quantiles of the given input data (not available when spline="p_spline" or "mono_p_spline")

- random_state: the random seed. default=0

# Case 1: Sine Ridge Function

In [12]:
s_star = 5
n_features = 100
n_samples = 10000

np.random.seed(1)
beta = np.zeros(n_features)
supp_ids = np.random.choice(n_features, s_star)
beta[supp_ids]=np.random.choice((-1, 1), s_star) / np.sqrt(s_star)

x = np.random.normal(0, 0.3, size=(n_samples, n_features))
y = np.sin(np.pi*(np.dot(x, beta))) + 0.1 * np.random.randn(n_samples)

The best hyperparameter combination can be selected via cross-validation

In [13]:
clf = SimRegressor(spline="smoothing_spline", knot_num=20, random_state=0)
clf.fit(x, y)

SimRegressor(degree=2, knot_dist='uniform', knot_num=20, method='first_order',
             random_state=0, reg_gamma=0.1, reg_lambda=0.1,
             spline='smoothing_spline')

In [15]:
clf.shape_fit_.sm_

R object with classes: ('smooth.spline',) mapped to:
[FloatSexp..., FloatSexp..., FloatSexp..., FloatSexp..., ..., IntSexpVe..., NULLType, ListSexpV..., LangSexpV...]
  x: <class 'numpy.ndarray'>
  array([-1.06289421, -1.06135891, -1.03578866, ...,  1.10245529,
        1.11442086,  1.16493218])
  y: <class 'numpy.ndarray'>
  array([-3.02778345, -3.00369896, -2.60823281, ..., -0.5870348 ,
       -0.41459095,  0.32053097])
  w: <class 'numpy.ndarray'>
  array([1., 1., 1., ..., 1., 1., 1.])
  yin: <class 'numpy.ndarray'>
  array([-3.32815353, -2.93789199, -2.49776022, ..., -0.62823895,
       -0.14723561,  0.07283352])
...
  data: <class 'numpy.ndarray'>
  array([          1,           1,         500,           0, -2147483648],
      dtype=int32)
  no.weights: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7fe1f4317848> [RTYPES.NILSXP]
R object with classes: ('smooth.spline',) mapped to:
[FloatSexp..., FloatSexp..., FloatSexp..., FloatSexp..., ..., IntSexpVe..., NULLType, ListSexpV..., LangSexpV...]
  cv.crit: <class 'rpy2.robjects.language.LangVector'>
  Rlang( (function (x, y = NULL, w = NULL, df, spar = NULL, lambda = NULL,  )

In [9]:

import rpy2
from rpy2 import robjects as ro
from rpy2.robjects import Formula
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri, pandas2ri

numpy2ri.activate()
pandas2ri.activate()

try:
    stats = importr("stats")
except:
    utils = importr('utils')
    utils.install_packages('stats', repos='http://cran.us.r-project.org')
    stats = importr("stats")

x = np.dot(x, beta)
n_samples = x.shape[0]
sample_weight = np.ones(n_samples)

unique_num = len(np.unique(x.round(decimals=6)))
if unique_num >= 4:
    y = y.copy() * 4 - 2
    clf.knot_num = min(unique_num, clf.knot_num)
    if clf.knot_dist == "uniform":
        clf.sm_ = stats.smooth_spline(x, y, nknots=clf.knot_num,
                             spar=clf.reg_gamma, w=sample_weight, tol=1e-6 * (np.max(x) - np.min(x)))
    elif clf.knot_dist == "quantile":
        knots = np.percentile(x, list(np.linspace(0, 100, clf.knot_num + 2, dtype=np.float32))).tolist()
        knots = (knots - clf.xmin) / (clf.xmax - clf.xmin)
        clf.sm_ = stats.smooth_spline(x, y, all_knots=ro.FloatVector(knots),
                             spar=clf.reg_gamma, w=sample_weight, tol=1e-6 * (np.max(x) - np.min(x)))
else:
    clf.sm_ = stats.glm(Formula('y ~ x'), family="gaussian",
            data=pd.DataFrame({"x":x.ravel(), "y":y.ravel()}),
            weights=sample_weight)


In [10]:
clf.sm_

R object with classes: ('smooth.spline',) mapped to:
[FloatSexp..., FloatSexp..., FloatSexp..., FloatSexp..., ..., IntSexpVe..., NULLType, ListSexpV..., LangSexpV...]
  x: <class 'numpy.ndarray'>
  array([-1.16329906, -1.12247435, -1.1024946 , ...,  1.04210111,
        1.06200675,  1.06630906])
  y: <class 'numpy.ndarray'>
  array([ -0.85616936,  -3.24208612,  -4.39779846, ..., -12.6372662 ,
       -13.83982754, -14.1026726 ])
  w: <class 'numpy.ndarray'>
  array([1., 1., 1., ..., 1., 1., 1.])
  yin: <class 'numpy.ndarray'>
  array([ -1.70866594,  -2.58894243,  -4.5129558 , ..., -11.9910409 ,
       -13.75156795, -15.31261412])
...
  data: <class 'numpy.ndarray'>
  array([          1,           1,         500,           0, -2147483648],
      dtype=int32)
  no.weights: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7fe1f4317848> [RTYPES.NILSXP]
R object with classes: ('smooth.spline',) mapped to:
[FloatSexp..., FloatSexp..., FloatSexp..., FloatSexp..., ..., IntSexpVe..., NULLType, ListSexpV..., LangSexpV...]
  cv.crit: <class 'rpy2.robjects.language.LangVector'>
  Rlang( (function (x, y = NULL, w = NULL, df, spar = NULL, lambda = NULL,  )

In [ ]:
%%time 

param_grid = {"method": ["first_order", "second_order"],
              "knot_dist": ["uniform", "quantile"],
              "reg_lambda": [0.1, 0.2, 0.3, 0.4, 0.5], 
              "reg_gamma": [0.2, 0.4, 0.6, 0.8, 1.0]}
grid = GridSearchCV(SimRegressor(spline="smoothing_spline", knot_num=20, random_state=0), iid=False,
                    cv=KFold(3, shuffle=True, random_state=0), param_grid=param_grid, n_jobs=-1, verbose=2, error_score=np.nan)
grid.fit(x, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.7s
R[write to console]: Error: ignoring SIGPIPE signal

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/share/R/library’ contains no packages

R[write to console]: 2: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/share/R/library’ contains no packages

R[write to console]: Fatal error: unable to initialize the JIT




In [ ]:
grid.best_params_

**NB**: The first-order Setin's method is selected, as $\mathbb{E}[f^{\prime\prime}(u)]=0$. Therefore, the second order method cannot provide any information about the projection coefficients. 

In this case, the first-order Setin's method is selected. 

In [ ]:
clf = grid.best_estimator_
clf

In [ ]:
plt.plot(np.abs(clf.beta_), "o")
plt.plot(np.abs(beta), "o")
plt.legend(["Estimated", "Ground Truth"])
plt.show()

In [ ]:
clf.visualize()

## Case 2: Quadratic Ridge Function

In [ ]:
s_star = 5
n_features = 100
n_samples = 10000

np.random.seed(1)
beta = np.zeros(n_features)
supp_ids = np.random.choice(n_features, s_star)
beta[supp_ids]=np.random.choice((-1, 1), s_star) / np.sqrt(s_star)

x = np.random.normal(0, 0.3, size=(n_samples, n_features))
y = np.dot(x, beta) ** 2 + 0.1 * np.random.randn(n_samples)

In [ ]:
%%time

param_grid = {"method": ["first_order", "second_order"],
              "knot_dist": ["uniform", "quantile"],
              "reg_lambda": [0.1, 0.2, 0.3, 0.4, 0.5], 
              "reg_gamma": [0.2, 0.4, 0.6, 0.8, 1.0]}
grid = GridSearchCV(SimRegressor(spline="smoothing_spline", knot_num=20, random_state=0), iid=False,
                    cv=KFold(3, shuffle=True, random_state=0), param_grid=param_grid, n_jobs=-1, verbose=2, error_score=np.nan)
grid.fit(x, y)

In [ ]:
grid.best_params_

**NB**: The second-order Setin's method is selected, as $\mathbb{E}[f^{\prime}(u)]=0$. Therefore, the first order method cannot provide any information about the projection coefficients. 

In [ ]:
clf = grid.best_estimator_
clf

In [ ]:
plt.plot(np.abs(clf.beta_), "o")
plt.plot(np.abs(beta), "o")
plt.legend(["Estimated", "Ground Truth"])
plt.show()

In [ ]:
clf.visualize()

## Case 3: Improve a rough estimator via inner update

In [ ]:
s_star = 5
n_features = 100
n_samples = 10000

np.random.seed(1)
beta = np.zeros(n_features)
supp_ids = np.random.choice(n_features, s_star)
beta[supp_ids]=np.random.choice((-1, 1), s_star) / np.sqrt(s_star)

x = np.random.gamma(1, 0.3, size=(n_samples, n_features))
y = np.sin(np.pi*(np.dot(x, beta))) + 0.1 * np.random.randn(n_samples)

In [ ]:
%%time

clf = SimRegressor(degree=2, knot_num=20, reg_lambda=0.1, reg_gamma=0.4, spline="smoothing_spline", random_state=0)
clf.fit(x, y)

In [ ]:
clf.visualize()

**Remark**：The data does not follow a normal distribution, so the performance is relatively poorer, we use adam optimizer to improve the estimation.

In [ ]:
clf.fit_inner_update(x, y, verbose=True, n_inner_iter_no_change=1)

**Remark**: fit_inner_update is not available for spline="p_spline" or "mono_p_spline"

In [ ]:
clf.visualize()